In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf
cf.go_offline(connected=True)
from tqdm import tqdm
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import pymysql

# 데이터셋 구축 & 파일저장 함수

In [1]:
def make_data_train(trading, days, target): 
    # 코드 리스트 
   
    IF=open("../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    
    # trainX, trainY
    lst_trainX = []
    lst_trainY = []
    lst_data = []
    lst_target = []
    
    # database
    con = pymysql.connect(
    )

    cursor = con.cursor()
 
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE Date
                    BETWEEN '2018-01-01' AND '2020-12-31'
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        lst_stock = stock.values.tolist()  


        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1]
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading]
                y = int(row[6] >= target)
                target = float(row[6])
                
                data = ','.join(map(str,lst_result))
                
                lst_trainX.append(data)
                lst_trainY.append(y)
                lst_data.append([code,date,data,y])
                lst_target.append(target)
    return np.array(lst_trainX), np.array(lst_trainY),np.array(lst_data),np.array(lst_target)

In [2]:
def make_data_test(trading, days, target): 
    # 코드 리스트 
   
    IF=open("../data/code_list.txt", 'r')
    lst_code = IF.readlines()
    

    lst_testX = []
    lst_testY = []
    lst_data = []
    lst_target = []
    
    # database
    con = pymysql.connect(

    )

    cursor = con.cursor()
 
    for line in tqdm(lst_code): 
        code = line.strip()
        sql_query = '''
                    SELECT *
                    FROM stock_{}
                    WHERE Date
                    BETWEEN '2021-01-01' AND '2021-06-30'
                    '''.format(code)
        stock = pd.read_sql(sql = sql_query, con = con)
        lst_stock = stock.values.tolist()  


        for i, row in enumerate(lst_stock): 
            date, close, volume = row[0].strftime('%Y%m%d'), row[4], row[5]
            # 거래대금, 이동평균선 범위 조건
            if (close * volume >= trading) :
                if i < (days-1):   # 예외 처리 
                    continue                               

                lst_result = []
                
                Ddays_date = lst_stock[i-(days-1):i+1]
                                           
                for row2 in Ddays_date:
                    Open,High,Low,Close,Volume = row2[1:6]
                    Trading = Close * Volume
                    lst_result += [Open,High,Low,Close,Trading]
                y = int(row[6] >= target)
                target = float(row[6])
                data = ','.join(map(str,lst_result))
                
                lst_testX.append(data)
                lst_testY.append(y)
                lst_data.append([code,date,data,y])
                lst_target.append(target)
    return np.array(lst_testX), np.array(lst_testY),np.array(lst_data),np.array(lst_target)

In [54]:
def dump_file(TYPE, trading, days,target,data):
    with open('../data/{}_{}_{}_{}.pickle'.format(TYPE,trading, days,target), 'wb') as f:
        pickle.dump(data, f)

In [102]:
def load_file(TYPE,trading, days,target):
    with open('../data/{}_{}_{}_{}.pickle'.format(TYPE,trading, days,target), 'rb') as f:
        data = pickle.load(f)
    return data

# Change 0.02
## 거래대금 1조 

In [65]:
trainX, trainY,train_data,train_target= make_data_train(1000000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:06<00:00, 23.49it/s]


In [66]:
testX, testY,test_data,test_target= make_data_test(1000000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:28<00:00, 55.61it/s]


In [67]:
dump_file('train',1000000000000, 10, 0.02,train_data)
dump_file('test',1000000000000, 10, 0.02,test_data)
dump_file('trainX',1000000000000, 10, 0.02,trainX)
dump_file('trainY',1000000000000, 10, 0.02,trainY)
dump_file('testX',1000000000000, 10, 0.02,testX)
dump_file('testY',1000000000000, 10, 0.02,testY)


## 거래대금 1천억 

In [68]:
trainX, trainY,train_data,train_target= make_data_train(100000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:12<00:00, 21.51it/s]


In [69]:
testX, testY,test_data,test_target= make_data_test(100000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:25<00:00, 62.20it/s]


In [70]:
dump_file('train',100000000000, 10, 0.02,train_data)
dump_file('test',100000000000, 10, 0.02,test_data)
dump_file('trainX',100000000000, 10, 0.02,trainX)
dump_file('trainY',100000000000, 10, 0.02,trainY)
dump_file('testX',100000000000, 10, 0.02,testX)
dump_file('testY',100000000000, 10, 0.02,testY)

## 거래대금 100억 

In [71]:
trainX, trainY,train_data,train_target= make_data_train(10000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:10<00:00, 22.08it/s]


In [72]:
testX, testY,test_data,test_target= make_data_test(10000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:24<00:00, 62.93it/s]


In [73]:
dump_file('train',10000000000, 10, 0.02,train_data)
dump_file('test',10000000000, 10, 0.02,test_data)
dump_file('trainX',10000000000, 10, 0.02,trainX)
dump_file('trainY',10000000000, 10, 0.02,trainY)
dump_file('testX',10000000000, 10, 0.02,testX)
dump_file('testY',10000000000, 10, 0.02,testY)

## 거래대금 10억

In [74]:
trainX, trainY,train_data,train_target= make_data_train(1000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:19<00:00, 19.65it/s]


In [75]:
testX, testY,test_data,test_target= make_data_test(1000000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:42<00:00, 37.02it/s]


In [76]:
dump_file('train',1000000000, 10, 0.02,train_data)
dump_file('test',1000000000, 10, 0.02,test_data)
dump_file('trainX',1000000000, 10, 0.02,trainX)
dump_file('trainY',1000000000, 10, 0.02,trainY)
dump_file('testX',1000000000, 10, 0.02,testX)
dump_file('testY',1000000000, 10, 0.02,testY)

## 거래대금 1억

In [77]:
trainX, trainY,train_data,train_target= make_data_train(100000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:30<00:00, 17.26it/s]


In [78]:
testX, testY,test_data,test_target= make_data_test(100000000, 10, 0.02)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:32<00:00, 47.79it/s]


In [79]:
dump_file('train',100000000, 10, 0.02,train_data)
dump_file('test',100000000, 10, 0.02,test_data)
dump_file('trainX',100000000, 10, 0.02,trainX)
dump_file('trainY',100000000, 10, 0.02,trainY)
dump_file('testX',100000000, 10, 0.02,testX)
dump_file('testY',100000000, 10, 0.02,testY)

# Change 0.05
## 거래대금 1조  

In [80]:
trainX, trainY,train_data,train_target= make_data_train(1000000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [02:19<00:00, 11.17it/s]


In [81]:
testX, testY,test_data,test_target= make_data_test(1000000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:26<00:00, 58.34it/s]


In [82]:
dump_file('train',1000000000000, 10, 0.05,train_data)
dump_file('test',1000000000000, 10, 0.05,test_data)
dump_file('trainX',1000000000000, 10, 0.05,trainX)
dump_file('trainY',1000000000000, 10, 0.05,trainY)
dump_file('testX',1000000000000, 10, 0.05,testX)
dump_file('testY',1000000000000, 10, 0.05,testY)


## 거래대금 1천억 

In [83]:
trainX, trainY,train_data,train_target= make_data_train(100000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:21<00:00, 19.22it/s]


In [84]:
testX, testY,test_data,test_target= make_data_test(100000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:36<00:00, 42.38it/s]


In [85]:
dump_file('train',100000000000, 10, 0.05,train_data)
dump_file('test',100000000000, 10, 0.05,test_data)
dump_file('trainX',100000000000, 10, 0.05,trainX)
dump_file('trainY',100000000000, 10, 0.05,trainY)
dump_file('testX',100000000000, 10, 0.05,testX)
dump_file('testY',100000000000, 10, 0.05,testY)

## 거래대금 100억 

In [86]:
trainX, trainY,train_data,train_target= make_data_train(10000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:18<00:00, 19.99it/s]


In [87]:
testX, testY,test_data,test_target= make_data_test(10000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:28<00:00, 55.42it/s]


In [88]:
dump_file('train',10000000000, 10, 0.05,train_data)
dump_file('test',10000000000, 10, 0.05,test_data)
dump_file('trainX',10000000000, 10, 0.05,trainX)
dump_file('trainY',10000000000, 10, 0.05,trainY)
dump_file('testX',10000000000, 10, 0.05,testX)
dump_file('testY',10000000000, 10, 0.05,testY)

## 거래대금 10억

In [89]:
trainX, trainY,train_data,train_target= make_data_train(1000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:21<00:00, 19.17it/s]


In [90]:
testX, testY,test_data,test_target= make_data_test(1000000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:37<00:00, 41.48it/s]


In [91]:
dump_file('train',1000000000, 10, 0.05,train_data)
dump_file('test',1000000000, 10, 0.05,test_data)
dump_file('trainX',1000000000, 10, 0.05,trainX)
dump_file('trainY',1000000000, 10, 0.05,trainY)
dump_file('testX',1000000000, 10, 0.05,testX)
dump_file('testY',1000000000, 10, 0.05,testY)

## 거래대금 1억

In [92]:
trainX, trainY,train_data,train_target= make_data_train(100000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [01:33<00:00, 16.75it/s]


In [93]:
testX, testY,test_data,test_target= make_data_test(100000000, 10, 0.05)

100%|██████████████████████████████████████████████████████████████████████████████| 1561/1561 [00:34<00:00, 45.90it/s]


In [94]:
dump_file('train',100000000, 10, 0.05,train_data)
dump_file('test',100000000, 10, 0.05,test_data)
dump_file('trainX',100000000, 10, 0.05,trainX)
dump_file('trainY',100000000, 10, 0.05,trainY)
dump_file('testX',100000000, 10, 0.05,testX)
dump_file('testY',100000000, 10, 0.05,testY)